# 完整版本项目梳理

## 01 树模型

### 1.1 环境设置与导入

In [72]:
import os
import sys
import json
from functools import partial
from datetime import datetime
from hashlib import md5
from typing import Any, Dict, List, Union
import pandas as pd
import yaml
import numpy as np
from pprint import pprint
from glob import glob  # 添加glob模块导入
import time
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [73]:
# 添加项目路径
project_root = os.getcwd()
env_path = os.path.join(project_root, 'env')
if env_path not in sys.path:
    sys.path.insert(0, env_path)

### 1.2 加载原始数据

In [74]:
# 02 根据配置文件加载原始数据
def load_raw_data_from_config():
    """根据data.yml配置文件加载原始数据，支持本地和线上环境"""
    
    # 加载数据配置
    with open('config/data.yml', 'r', encoding='utf-8') as f:
        data_config = yaml.safe_load(f)
    
    # 检测文件类型和环境
    train_dir = data_config['train_dir']
    
    # 检查CSV和TXT文件
    csv_files = glob(os.path.join(train_dir, '*.csv'))
    txt_files = glob(os.path.join(train_dir, '*.txt'))
    
    if csv_files:
        # 本地环境 - 使用CSV格式
        print("检测到CSV文件，使用本地环境配置")
        csv_config = data_config['csv_format']
        separator, header = csv_config['separator'], csv_config['header']
        
        print(f"分隔符: '{separator}', 表头行: {header}, 文件数量: {len(csv_files)}")
        
        # 读取CSV文件
        dfs = [pd.read_csv(f, sep=separator, header=header) for f in csv_files]
        
    elif txt_files:
        # 线上环境 - 使用TXT格式
        print("检测到TXT文件，使用线上环境配置")
        txt_config = data_config.get('txt_format', {'separator': '\t', 'header': None})
        separator, header = txt_config['separator'], txt_config['header']
        
        # 从列表中提取列名
        raw_columns = [list(item.keys())[0] for item in data_config['raw_data_columns']]
        
        print(f"分隔符: '{separator}', 表头行: {header}, 文件数量: {len(txt_files)}")
        print(f"预定义列名: {raw_columns}")
        
        # 读取TXT文件
        dfs = [pd.read_csv(f, sep=separator, header=header, names=raw_columns) for f in txt_files]
    else:
        raise ValueError(f"在目录 {train_dir} 中未找到CSV或TXT文件")
    
    df_raw = pd.concat(dfs, ignore_index=True)
    print(f"形状{df_raw.shape}, 列名: {list(df_raw.columns)}")
    
    return df_raw

# 加载原始数据
df_raw = load_raw_data_from_config()

# 显示数据样例（简化版）
print(df_raw.iloc[0].to_dict())

检测到CSV文件，使用本地环境配置
分隔符: ',', 表头行: 0, 文件数量: 5
形状(50000, 13), 列名: ['user_id', 'create_time', 'log_type', 'watchlists', 'holdings', 'country', 'prefer_bid', 'user_propernoun', 'push_title', 'push_content', 'item_code', 'item_tags', 'submit_type']
{'user_id': 1800001088, 'create_time': '2025-05-31 08:39:07', 'log_type': 'PR', 'watchlists': nan, 'holdings': nan, 'country': 'Germany', 'prefer_bid': nan, 'user_propernoun': 'germany#3.06|mid-america#1.02', 'push_title': 'Ainvest Newswire', 'push_content': 'Hims & Hers Health Lays Off 4% of Staff Amid Strategy Shift', 'item_code': '[{"market":"169","score":0,"code":"HIMS","tagId":"U000012934","name":"Hims & Hers Health","type":0,"parentId":"US_ROBOT0f37d7fd3fca6a41"},{"market":"169","score":0,"code":"NVO","tagId":"U000002999","name":"Novo Nordisk","type":0,"parentId":"US_ROBOT0f37d7fd3fca6a41"}]', 'item_tags': '[{"score":0.7803922295570374,"tagId":"51510","name":"us_high_importance","type":4,"parentId":"US_ROBOT0f37d7fd3fca6a41"},{"score":0.7803

### 1.3 加载并解析YAML配置

In [75]:
# 加载树模型配置 (config.yml 包含特征和训练配置)
with open('config/config.yml', 'r', encoding='utf-8') as f:
    config_yml = yaml.safe_load(f)
# 从 config.yml 提取树模型的特征配置
tree_feat_config = config_yml['features']
# 加载深度模型的特征配置 (feat.yml)
with open('config/feat.yml', 'r', encoding='utf-8') as f:
    deep_feat_config = yaml.safe_load(f)

# 当前使用树模型的特征配置进行演示
feat_config = tree_feat_config

print(feat_config)

{'process': {'embedding_dim': 8, 'pooling_type': 'sum', 'pipelines': [{'feat_name': 'hour', 'feat_type': 'sparse', 'vocabulary_size': 24, 'embedding_dim': 8, 'input_sample': '2024-08-02 00:44:05', 'operations': [{'col_in': 'create_time', 'col_out': 'create_time', 'func_name': 'fillna', 'func_parameters': {'na_value': '2024-08-02 00:16:34'}}, {'col_in': 'create_time', 'col_out': 'hour', 'func_name': 'to_hour', 'func_parameters': {}}]}, {'feat_name': 'weekday', 'feat_type': 'sparse', 'vocabulary_size': 7, 'embedding_dim': 8, 'input_sample': '2024-08-02 00:44:05', 'operations': [{'col_in': 'create_time', 'col_out': 'weekday', 'func_name': 'to_weekday', 'func_parameters': {}}]}, {'feat_name': 'user_watch_stk_code_hash', 'feat_type': 'varlen_sparse', 'vocabulary_size': 10000, 'embedding_dim': 8, 'input_sample': 'AAPL_185 & TSLA_185', 'operations': [{'col_in': 'watchlists', 'col_out': 'watchlists', 'func_name': 'fillna', 'func_parameters': {'na_value': 'null_0 & null_0'}}, {'col_in': 'watchl

In [76]:
# 看一下一个pipeline中对一个特征的操作 他被解析成了什么结构
# 适配 config.yml 和 feat.yml 的不同结构
if 'pipelines' in feat_config:
    # feat.yml 格式
    example_pipeline = feat_config['pipelines'][0]
elif 'process' in feat_config and 'pipelines' in feat_config['process']:
    # config.yml 格式
    example_pipeline = feat_config['process']['pipelines'][0]
else:
    print("无法找到特征配置格式")
    example_pipeline = None

if example_pipeline:
    pprint(example_pipeline)

{'embedding_dim': 8,
 'feat_name': 'hour',
 'feat_type': 'sparse',
 'input_sample': '2024-08-02 00:44:05',
 'operations': [{'col_in': 'create_time',
                 'col_out': 'create_time',
                 'func_name': 'fillna',
                 'func_parameters': {'na_value': '2024-08-02 00:16:34'}},
                {'col_in': 'create_time',
                 'col_out': 'hour',
                 'func_name': 'to_hour',
                 'func_parameters': {}}],
 'vocabulary_size': 24}


### 1.4 进行数据处理

#### 1.4.1 定义原子操作函数

In [77]:
MISSING_VALUE = [None, '', 'null', 'NULL', 'None', np.nan]

def fillna(x: Union[float, int, str], na_value: Union[float, int, str]) -> Union[float, int, str]:
    """填充缺失值"""
    if x in MISSING_VALUE or (isinstance(x, float) and pd.isna(x)):
        return na_value
    return x

def split(x: str, sep: str) -> List[str]:
    """字符串分割"""
    return str(x).split(sep)

def seperation(x: List[str], sep: str) -> List[List[str]]:
    """列表元素二次分割"""
    if not isinstance(x, list):
        return []
    return [item.split(sep) for item in x]

def list_get(x: List[List[Any]], item_index: int) -> List[Any]:
    """获取嵌套列表中指定位置的元素"""
    if not isinstance(x, list):
        return []
    result = []
    for sublist in x:
        if isinstance(sublist, list) and len(sublist) > item_index:
            result.append(sublist[item_index])
        else:
            result.append('null')
    return result

def remove_items(x: List[str], target_values: List[str]) -> List[str]:
    """移除列表中的指定元素"""
    if not isinstance(x, list):
        return []
    return [item for item in x if item not in target_values]

def padding(x: List[Any], pad_value: Union[str, float, int], max_len: int) -> List[Any]:
    """列表填充到指定长度"""
    if not isinstance(x, list):
        x = []
    if len(x) >= max_len:
        return x[:max_len]
    else:
        return x + [pad_value] * (max_len - len(x))

def list_hash(x: List[str], vocabulary_size: int) -> List[int]:
    """对列表中每个元素进行哈希"""
    if not isinstance(x, list):
        return []
    result = []
    for item in x:
        hash_val = int(md5(str(item).encode()).hexdigest(), 16) % vocabulary_size
        result.append(hash_val)
    return result

def str_hash(x: str, vocabulary_size: int) -> int:
    """字符串哈希"""
    return int(md5(str(x).encode()).hexdigest(), 16) % vocabulary_size

def to_hour(x: str) -> int:
    """提取时间中的小时"""
    try:
        dt = pd.to_datetime(x)
        return dt.hour
    except:
        return 0

def to_weekday(x: str) -> int:
    """提取时间中的星期"""
    try:
        dt = pd.to_datetime(x)
        return dt.weekday()
    except:
        return 0

def list_len(x: List) -> int:
    """列表长度"""
    if isinstance(x, list):
        return len(x)
    return 0

def int_max(x: int, max_value: int) -> int:
    """限制整数最大值"""
    return min(int(x), max_value)

def json_object_to_list(x: str, key: str) -> List[str]:
    """从JSON对象列表中提取指定键的值"""
    try:
        data = json.loads(x)
        if isinstance(data, list):
            return [item.get(key, 'null') for item in data if isinstance(item, dict)]
        return ['null']
    except:
        return ['null']

def map_to_int(x: Union[str, List], map_dict: Dict[str, int], default_code: int = 0) -> Union[List[int], int]:
    """映射到整数"""
    if isinstance(x, list):
        return [map_dict.get(item, default_code) for item in x]
    else:
        return map_dict.get(str(x), default_code)

# 构建操作中心 (OP_HUB)
OP_HUB = {
    'fillna': fillna,
    'split': split,
    'seperation': seperation,
    'list_get': list_get,
    'remove_items': remove_items,
    'padding': padding,
    'list_hash': list_hash,
    'str_hash': str_hash,
    'to_hour': to_hour,
    'to_weekday': to_weekday,
    'list_len': list_len,
    'int_max': int_max,
    'json_object_to_list': json_object_to_list,
    'map_to_int': map_to_int
}

print(f"OP_HUB 构建完成，包含 {len(OP_HUB)} 个操作函数")
print(f"可用函数: {list(OP_HUB.keys())}")


OP_HUB 构建完成，包含 14 个操作函数
可用函数: ['fillna', 'split', 'seperation', 'list_get', 'remove_items', 'padding', 'list_hash', 'str_hash', 'to_hour', 'to_weekday', 'list_len', 'int_max', 'json_object_to_list', 'map_to_int']


#### 1.4.2 实现原子操作作用与df

In [78]:
def run_one_op(df: pd.DataFrame, operation: dict) -> pd.DataFrame:
    """执行单个特征操作"""
    # 获取操作配置
    col_in = operation['col_in']
    col_out = operation['col_out']
    func_name = operation['func_name']
    parameters = operation.get('func_parameters', {})
    
    # 检查函数是否存在
    if func_name not in OP_HUB:
        return df
    
    # 检查输入列是否存在
    input_cols = [col_in] if isinstance(col_in, str) else col_in
    if not all(col in df.columns for col in input_cols):
        return df
    
    # 准备特征转换函数
    transform_func = partial(OP_HUB[func_name], **parameters)
    
    # 执行特征转换
    if isinstance(col_in, list):
        df[col_out] = df[col_in].apply(lambda row: transform_func(*row), axis=1)
    else:
        df[col_out] = df[col_in].apply(transform_func)
    
    return df

#### 1.4.3 实现原子操作拼接成完整的process函数作用与df

In [79]:
def process_feature_pipelines(df_raw: pd.DataFrame, feat_config: dict) -> tuple[pd.DataFrame, list]:
    """执行特征工程流水线 - 适配不同的配置文件格式"""
    # 创建数据副本
    df = df_raw.copy()
    
    # 获取需要处理的流水线 - 适配不同格式
    if 'pipelines' in feat_config:
        # feat.yml 格式: { pipelines: [...] }
        pipelines = feat_config['pipelines']
    elif 'process' in feat_config and 'pipelines' in feat_config['process']:
        # config.yml 格式: { process: { pipelines: [...] } }
        pipelines = feat_config['process']['pipelines']
    else:
        print("⚠️ 无法找到特征配置中的 pipelines")
        return df, []

    # 记录成功处理的特征
    processed_features = []
    
    # 执行每个特征处理流水线
    for pipeline in pipelines:
        feat_name = pipeline['feat_name']
        operations = pipeline['operations']
        
        # 执行流水线中的每个操作
        for operation in operations:
            df = run_one_op(df, operation)

        # 记录处理成功的特征
        processed_features.append(feat_name)
    
    return df, processed_features

df_processed, processed_features = process_feature_pipelines(df_raw, feat_config)

### 1.5 特征处理后数据集分析

In [80]:
print("数据结构对比:")
print(f"原始列数: {len(df_raw.columns)}")
print(f"处理后列数: {len(df_processed.columns)}")
print(f"新增列数: {len(df_processed.columns) - len(df_raw.columns)}")

print("\n原始列名:")
print(list(df_raw.columns))

print("\n新增列名:")
new_columns = [col for col in df_processed.columns if col not in df_raw.columns]
print(new_columns)

# 查看成功生成的特征
print("成功生成的特征详情:")
for feat_name in processed_features:
    if feat_name in df_processed.columns:
        sample_data = df_processed[feat_name].iloc[0]
        data_type = type(sample_data).__name__
        print(f"  {feat_name}: {data_type} = {sample_data}")

print("最终处理结果预览:")
display_cols = ['user_id', 'log_type'] + processed_features
display_cols = [col for col in display_cols if col in df_processed.columns]

print(df_processed[display_cols].head())

数据结构对比:
原始列数: 13
处理后列数: 30
新增列数: 17

原始列名:
['user_id', 'create_time', 'log_type', 'watchlists', 'holdings', 'country', 'prefer_bid', 'user_propernoun', 'push_title', 'push_content', 'item_code', 'item_tags', 'submit_type']

新增列名:
['hour', 'weekday', 'user_watch_stk_code', 'user_watch_stk_code_hash', 'country_hash', 'prefer_bid_code', 'prefer_bid_code_hash', 'hold_bid_code', 'hold_bid_code_hash', 'user_propernoun_code', 'user_propernoun_hash', 'push_title_hash', 'title_len', 'item_code_hash', 'submit_type_hash', 'tagIds', 'tag_id_hash']
成功生成的特征详情:
  hour: int64 = 8
  weekday: int64 = 5
  user_watch_stk_code_hash: list = [8381, 8381, 8381, 8381, 8381]
  country_hash: int64 = 71
  prefer_bid_code_hash: list = [8381, 8381, 8381, 8381, 8381]
  hold_bid_code_hash: list = [8381, 8381, 8381, 8381, 8381]
  user_propernoun_hash: list = [178, 417, 8381, 8381, 8381]
  push_title_hash: int64 = 7
  title_len: int64 = 12
  item_code_hash: list = [6837, 3491, 8381, 8381, 8381]
  submit_type_hash: int6

### 1.6 模型训练与评估

In [81]:
def prepare_features(df_processed, processed_features, max_list_length=5):
    """展开列表特征 树模型的需求"""
    df_tree = df_processed[processed_features].copy()
    
    for feat in processed_features:
        if isinstance(df_tree[feat].iloc[0], list):
            expanded = df_tree[feat].apply(pd.Series).iloc[:, :max_list_length]
            expanded.columns = [f"{feat}_{i}" for i in range(expanded.shape[1])]
            df_tree = df_tree.drop(columns=[feat]).join(expanded)
    
    return df_tree

def train_model(X, y, train_params):
    """训练LightGBM模型"""
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    train_data = lgb.Dataset(X_train, y_train)
    val_data = lgb.Dataset(X_val, y_val, reference=train_data)
    
    model = lgb.train(
        train_params,
        train_data,
        num_boost_round=train_params.pop('num_iterations', 1000),
        callbacks=[lgb.early_stopping(train_params.pop('early_stopping_rounds', 100))],
        valid_sets=[train_data, val_data],
        valid_names=['train', 'valid']
    )
    
    return model, X_train, X_val, y_train, y_val

def evaluate_model(model, X_train, X_val, y_train, y_val):
    """评估模型性能"""
    y_train_pred = model.predict(X_train, num_iteration=model.best_iteration)
    y_val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    
    train_auc = roc_auc_score(y_train, y_train_pred)
    val_auc = roc_auc_score(y_val, y_val_pred)
    
    print(f"训练集 AUC: {train_auc:.4f}")
    print(f"验证集 AUC: {val_auc:.4f}")
    
    return pd.DataFrame({
        'feature': model.feature_name(),
        'importance': model.feature_importance(importance_type='gain')
    }).sort_values('importance', ascending=False)

In [82]:
# 树模型使用config.yml配置
with open('config/config.yml', 'r', encoding='utf-8') as f:
    tree_config = yaml.safe_load(f)

# 准备数据
df_processed['label'] = df_processed['log_type'].apply(lambda x: 1 if x == 'PC' else 0)
X = prepare_features(df_processed, processed_features)
y = df_processed['label']

# 训练模型
train_params = {**tree_config['train'], 'verbose': -1, 'n_jobs': -1, 'seed': 42}
model, X_train, X_val, y_train, y_val = train_model(X, y, train_params)

# 评估并输出结果
feature_importance = evaluate_model(model, X_train, X_val, y_train, y_val)
print("\n特征重要性 (Top 20):")
print(feature_importance.head(20))

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[300]	train's auc: 0.864733	valid's auc: 0.847046
训练集 AUC: 0.8647
验证集 AUC: 0.8470

特征重要性 (Top 20):
                       feature     importance
23      user_propernoun_hash_2  346798.388273
21      user_propernoun_hash_0  115543.097794
2                 country_hash   60724.277898
22      user_propernoun_hash_1   55430.613762
1                      weekday   23594.930362
0                         hour   10044.143828
32               tag_id_hash_1    5167.244439
5             submit_type_hash    4476.428500
10  user_watch_stk_code_hash_4    4473.375479
31               tag_id_hash_0    4001.311887
33               tag_id_hash_2    3325.424608
4                    title_len    3310.380234
6   user_watch_stk_code_hash_0    3213.638852
3              push_title_hash    3065.194983
7   user_watch_stk_code_hash_1    2862.350930
8   user_watch_stk_code_hash_2    2614.035170
9   user_w

## 02 MLP模型

### 2.1 环境设置与导入

In [83]:
import tensorflow as tf
from keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, BatchNormalization, Concatenate
from keras.models import Model
from keras.regularizers import l2

# 树模型时已经引入的
# import os
# import sys
# import json
# from functools import partial
# from datetime import datetime
# from hashlib import md5
# from typing import Any, Dict, List, Union
# import pandas as pd
# import yaml
# import numpy as np
# from pprint import pprint
# from glob import glob  # 添加glob模块导入
# import time
# import lightgbm as lgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_auc_score

### 2.2 加载原始数据

In [84]:
# 02 根据配置文件加载原始数据
# 和树模型部分一样
def load_raw_data_from_config():
    """根据data.yml配置文件加载原始数据，支持本地和线上环境"""
    
    # 加载数据配置
    with open('config/data.yml', 'r', encoding='utf-8') as f:
        data_config = yaml.safe_load(f)
    
    # 检测文件类型和环境
    train_dir = data_config['train_dir']
    
    # 检查CSV和TXT文件
    csv_files = glob(os.path.join(train_dir, '*.csv'))
    txt_files = glob(os.path.join(train_dir, '*.txt'))
    
    if csv_files:
        # 本地环境 - 使用CSV格式
        print("检测到CSV文件，使用本地环境配置")
        csv_config = data_config['csv_format']
        separator, header = csv_config['separator'], csv_config['header']
        
        print(f"分隔符: '{separator}', 表头行: {header}, 文件数量: {len(csv_files)}")
        
        # 读取CSV文件
        dfs = [pd.read_csv(f, sep=separator, header=header) for f in csv_files]
        
    elif txt_files:
        # 线上环境 - 使用TXT格式
        print("检测到TXT文件，使用线上环境配置")
        txt_config = data_config.get('txt_format', {'separator': '\t', 'header': None})
        separator, header = txt_config['separator'], txt_config['header']
        
        # 从列表中提取列名
        raw_columns = [list(item.keys())[0] for item in data_config['raw_data_columns']]
        
        print(f"分隔符: '{separator}', 表头行: {header}, 文件数量: {len(txt_files)}")
        print(f"预定义列名: {raw_columns}")
        
        # 读取TXT文件
        dfs = [pd.read_csv(f, sep=separator, header=header, names=raw_columns) for f in txt_files]
    else:
        raise ValueError(f"在目录 {train_dir} 中未找到CSV或TXT文件")
    
    df_raw = pd.concat(dfs, ignore_index=True)
    print(f"形状{df_raw.shape}, 列名: {list(df_raw.columns)}")
    
    return df_raw

# 加载原始数据
df_raw = load_raw_data_from_config()

# 显示数据样例（简化版）
print(df_raw.iloc[0].to_dict())

检测到CSV文件，使用本地环境配置
分隔符: ',', 表头行: 0, 文件数量: 5
形状(50000, 13), 列名: ['user_id', 'create_time', 'log_type', 'watchlists', 'holdings', 'country', 'prefer_bid', 'user_propernoun', 'push_title', 'push_content', 'item_code', 'item_tags', 'submit_type']
{'user_id': 1800001088, 'create_time': '2025-05-31 08:39:07', 'log_type': 'PR', 'watchlists': nan, 'holdings': nan, 'country': 'Germany', 'prefer_bid': nan, 'user_propernoun': 'germany#3.06|mid-america#1.02', 'push_title': 'Ainvest Newswire', 'push_content': 'Hims & Hers Health Lays Off 4% of Staff Amid Strategy Shift', 'item_code': '[{"market":"169","score":0,"code":"HIMS","tagId":"U000012934","name":"Hims & Hers Health","type":0,"parentId":"US_ROBOT0f37d7fd3fca6a41"},{"market":"169","score":0,"code":"NVO","tagId":"U000002999","name":"Novo Nordisk","type":0,"parentId":"US_ROBOT0f37d7fd3fca6a41"}]', 'item_tags': '[{"score":0.7803922295570374,"tagId":"51510","name":"us_high_importance","type":4,"parentId":"US_ROBOT0f37d7fd3fca6a41"},{"score":0.7803

### 2.3 加载并解析YAML配置

In [91]:
# 加载深度模型的训练和模型配置（train.yml）
with open('config/train.yml', 'r', encoding='utf-8') as f:
    train_config = yaml.safe_load(f)

training_config = train_config.get('training', {})
model_config = train_config.get('model', {})

# 加载深度模型的特征配置 (feat.yml)
with open('config/feat.yml', 'r', encoding='utf-8') as f:
    deep_feat_config = yaml.safe_load(f)

print(training_config)
print(model_config)
print(deep_feat_config)

{'batch_size': 256, 'epochs': 2, 'lr': 0.0005, 'weight_decay': 0.001, 'lr_scheduler': {'type': 'ExponentialDecay', 'decay_steps': 100, 'decay_rate': 0.96, 'staircase': True, 'min_lr': 1e-05}, 'early_stopping': {'monitor': 'val_auc', 'patience': 2, 'mode': 'max', 'restore_best_weights': True}, 'reduce_lr': {'monitor': 'val_loss', 'factor': 0.5, 'patience': 1, 'min_lr': 1e-05}, 'shuffle_buffer_size': 20000, 'prefetch': 'auto', 'save_model': True, 'model_path': './models/push_binary_classification_model.keras', 'best_model_path': './models/best_model.keras', 'save_best_only': True, 'validation_split': 0.2}
{'layers': [128, 64, 32], 'dropout_rates': [0.3, 0.3, 0.2], 'l2_regularization': 0.001}
{'exclude_features': {'current': 'default', 'default': [], 'exclude_user_behavior': ['user_watch_stk_code', 'prefer_bid_code', 'hold_bid_code', 'user_propernoun'], 'exclude_user_propernoun': ['user_propernoun']}, 'pipelines': [{'embedding_dim': 8, 'feat_name': 'hour', 'feat_type': 'sparse', 'input_sa

### 2.4 进行数据处理

#### 2.4.1 定义原子操作函数

In [86]:
MISSING_VALUE = [None, '', 'null', 'NULL', 'None', np.nan]

def fillna(x: Union[float, int, str], na_value: Union[float, int, str]) -> Union[float, int, str]:
    """填充缺失值"""
    if x in MISSING_VALUE or (isinstance(x, float) and pd.isna(x)):
        return na_value
    return x

def split(x: str, sep: str) -> List[str]:
    """字符串分割"""
    return str(x).split(sep)

def seperation(x: List[str], sep: str) -> List[List[str]]:
    """列表元素二次分割"""
    if not isinstance(x, list):
        return []
    return [item.split(sep) for item in x]

def list_get(x: List[List[Any]], item_index: int) -> List[Any]:
    """获取嵌套列表中指定位置的元素"""
    if not isinstance(x, list):
        return []
    result = []
    for sublist in x:
        if isinstance(sublist, list) and len(sublist) > item_index:
            result.append(sublist[item_index])
        else:
            result.append('null')
    return result

def remove_items(x: List[str], target_values: List[str]) -> List[str]:
    """移除列表中的指定元素"""
    if not isinstance(x, list):
        return []
    return [item for item in x if item not in target_values]

def padding(x: List[Any], pad_value: Union[str, float, int], max_len: int) -> List[Any]:
    """列表填充到指定长度"""
    if not isinstance(x, list):
        x = []
    if len(x) >= max_len:
        return x[:max_len]
    else:
        return x + [pad_value] * (max_len - len(x))

def list_hash(x: List[str], vocabulary_size: int) -> List[int]:
    """对列表中每个元素进行哈希"""
    if not isinstance(x, list):
        return []
    result = []
    for item in x:
        hash_val = int(md5(str(item).encode()).hexdigest(), 16) % vocabulary_size
        result.append(hash_val)
    return result

def str_hash(x: str, vocabulary_size: int) -> int:
    """字符串哈希"""
    return int(md5(str(x).encode()).hexdigest(), 16) % vocabulary_size

def to_hour(x: str) -> int:
    """提取时间中的小时"""
    try:
        dt = pd.to_datetime(x)
        return dt.hour
    except:
        return 0

def to_weekday(x: str) -> int:
    """提取时间中的星期"""
    try:
        dt = pd.to_datetime(x)
        return dt.weekday()
    except:
        return 0

def list_len(x: List) -> int:
    """列表长度"""
    if isinstance(x, list):
        return len(x)
    return 0

def int_max(x: int, max_value: int) -> int:
    """限制整数最大值"""
    return min(int(x), max_value)

def json_object_to_list(x: str, key: str) -> List[str]:
    """从JSON对象列表中提取指定键的值"""
    try:
        data = json.loads(x)
        if isinstance(data, list):
            return [item.get(key, 'null') for item in data if isinstance(item, dict)]
        return ['null']
    except:
        return ['null']

def map_to_int(x: Union[str, List], map_dict: Dict[str, int], default_code: int = 0) -> Union[List[int], int]:
    """映射到整数"""
    if isinstance(x, list):
        return [map_dict.get(item, default_code) for item in x]
    else:
        return map_dict.get(str(x), default_code)

# 构建操作中心 (OP_HUB)
OP_HUB = {
    'fillna': fillna,
    'split': split,
    'seperation': seperation,
    'list_get': list_get,
    'remove_items': remove_items,
    'padding': padding,
    'list_hash': list_hash,
    'str_hash': str_hash,
    'to_hour': to_hour,
    'to_weekday': to_weekday,
    'list_len': list_len,
    'int_max': int_max,
    'json_object_to_list': json_object_to_list,
    'map_to_int': map_to_int
}

print(f"OP_HUB 构建完成，包含 {len(OP_HUB)} 个操作函数")
print(f"可用函数: {list(OP_HUB.keys())}")

OP_HUB 构建完成，包含 14 个操作函数
可用函数: ['fillna', 'split', 'seperation', 'list_get', 'remove_items', 'padding', 'list_hash', 'str_hash', 'to_hour', 'to_weekday', 'list_len', 'int_max', 'json_object_to_list', 'map_to_int']


#### 2.4.2 实现原子操作作用于df

In [87]:
def run_one_op(df: pd.DataFrame, operation: dict) -> pd.DataFrame:
    """执行单个特征操作"""
    # 获取操作配置
    col_in = operation['col_in']
    col_out = operation['col_out']
    func_name = operation['func_name']
    parameters = operation.get('func_parameters', {})
    
    # 检查函数是否存在
    if func_name not in OP_HUB:
        return df
    
    # 检查输入列是否存在
    input_cols = [col_in] if isinstance(col_in, str) else col_in
    if not all(col in df.columns for col in input_cols):
        return df
    
    # 准备特征转换函数
    transform_func = partial(OP_HUB[func_name], **parameters)
    
    # 执行特征转换
    if isinstance(col_in, list):
        df[col_out] = df[col_in].apply(lambda row: transform_func(*row), axis=1)
    else:
        df[col_out] = df[col_in].apply(transform_func)
    
    return df

#### 2.4.3 实现原子操作拼接成完整的process函数作用于df

In [88]:
def process_feature_pipelines(df_raw: pd.DataFrame, feat_config: dict) -> tuple[pd.DataFrame, list]:
    """执行特征工程流水线 - 适配不同的配置文件格式"""
    # 创建数据副本
    df = df_raw.copy()
    
    # 获取需要处理的流水线 - 适配不同格式
    if 'pipelines' in feat_config:
        # feat.yml 格式: { pipelines: [...] }
        pipelines = feat_config['pipelines']
    elif 'process' in feat_config and 'pipelines' in feat_config['process']:
        # config.yml 格式: { process: { pipelines: [...] } }
        pipelines = feat_config['process']['pipelines']
    else:
        print("⚠️ 无法找到特征配置中的 pipelines")
        return df, []

    # 记录成功处理的特征
    processed_features = []
    
    # 执行每个特征处理流水线
    for pipeline in pipelines:
        feat_name = pipeline['feat_name']
        operations = pipeline['operations']
        
        # 执行流水线中的每个操作
        for operation in operations:
            df = run_one_op(df, operation)

        # 记录处理成功的特征
        processed_features.append(feat_name)
    
    return df, processed_features

In [99]:
# label的处理
if 'label' not in df_raw.columns:
    df_raw['label'] = df_raw['log_type'].apply(lambda x: 1 if x == 'PC' else 0)

In [117]:
# 2.4节 只有这里是和1.4节不一样的
df_deep_processed, processed_features = process_feature_pipelines(df_raw, deep_feat_config)

### 2.5 特征处理后的数据分析

In [118]:
print("数据结构对比:")
print(f"原始列数: {len(df_raw.columns)}")
print(f"处理后列数: {len(df_deep_processed.columns)}")
print(f"新增列数: {len(df_deep_processed.columns) - len(df_raw.columns)}")

print("\n原始列名:")
print(list(df_raw.columns))

print("\n新增列名:")
new_columns = [col for col in df_deep_processed.columns if col not in df_raw.columns]
print(new_columns)

# 查看成功生成的特征
print("成功生成的特征详情:")
for feat_name in processed_features:
    if feat_name in df_deep_processed.columns:
        sample_data = df_deep_processed[feat_name].iloc[0]
        data_type = type(sample_data).__name__
        print(f"  {feat_name}: {data_type} = {sample_data}")

print("最终处理结果预览:")
display_cols = ['user_id', 'log_type'] + processed_features
display_cols = [col for col in display_cols if col in df_deep_processed.columns]

print(df_deep_processed[display_cols].head())

数据结构对比:
原始列数: 14
处理后列数: 31
新增列数: 17

原始列名:
['user_id', 'create_time', 'log_type', 'watchlists', 'holdings', 'country', 'prefer_bid', 'user_propernoun', 'push_title', 'push_content', 'item_code', 'item_tags', 'submit_type', 'label']

新增列名:
['hour', 'weekday', 'user_watch_stk_code', 'user_watch_stk_code_hash', 'country_hash', 'prefer_bid_code', 'prefer_bid_code_hash', 'hold_bid_code', 'hold_bid_code_hash', 'user_propernoun_code', 'user_propernoun_hash', 'push_title_hash', 'title_len', 'item_code_hash', 'submit_type_hash', 'tagIds', 'tag_id_hash']
成功生成的特征详情:
  hour: int64 = 8
  weekday: int64 = 5
  user_watch_stk_code_hash: list = [8381, 8381, 8381, 8381, 8381]
  country_hash: int64 = 71
  prefer_bid_code_hash: list = [8381, 8381, 8381, 8381, 8381]
  hold_bid_code_hash: list = [8381, 8381, 8381, 8381, 8381]
  user_propernoun_hash: list = [178, 417, 8381, 8381, 8381]
  push_title_hash: int64 = 7
  title_len: int64 = 12
  item_code_hash: list = [6837, 3491, 8381, 8381, 8381]
  submit_type_h

### 2.6 将pd数据集转为tf数据集

In [102]:
def prepare_tf_dataset_for_deep_model(df: pd.DataFrame, feat_config: dict, batch_size: int = 512) -> tf.data.Dataset:
    """准备TensorFlow数据集用于深度模型训练"""
    
    # 获取需要的特征 - 适配不同的配置文件格式
    if 'pipelines' in feat_config:
        # feat.yml 格式
        pipelines = feat_config['pipelines']
    elif 'process' in feat_config and 'pipelines' in feat_config['process']:
        # config.yml 格式
        pipelines = feat_config['process']['pipelines']
    else:
        raise ValueError("无法找到特征配置中的 pipelines")
    
    pipeline_feats = {p['feat_name']: p for p in pipelines}
    features_dict = {}
    
    for feat_name, config in pipeline_feats.items():
        if feat_name not in df.columns:
            continue
        
        feat_type = config.get('feat_type', 'sparse')
        
        if feat_type == 'sparse':
            # 单值特征，shape为 (batch_size,) - 这是关键！
            values = df[feat_name].values.astype(np.int32)
            features_dict[feat_name] = values
            
        elif feat_type == 'varlen_sparse':
            # 变长特征，需要padding
            sequences = df[feat_name].tolist()
            max_len = max(len(seq) if isinstance(seq, list) else 1 for seq in sequences)
            
            padded_sequences = []
            for seq in sequences:
                if isinstance(seq, list):
                    padded = seq + [0] * (max_len - len(seq))
                else:
                    padded = [int(seq)] + [0] * (max_len - 1)
                padded_sequences.append(padded[:max_len])
            
            features_dict[feat_name] = np.array(padded_sequences, dtype=np.int32)
            
        elif feat_type == 'dense':
            # 密集特征
            values = df[feat_name].values.astype(np.float32).reshape(-1, 1)
            features_dict[feat_name] = values
    
    # 准备标签
    labels = df['label'].values.astype(np.int32)
    
    # 创建数据集
    dataset = tf.data.Dataset.from_tensor_slices((features_dict, labels))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return dataset

batch_size = train_config.get('training', {}).get('batch_size', 256)  # 使用train.yml的默认值
full_dataset = prepare_tf_dataset_for_deep_model(df_deep_processed, deep_feat_config, batch_size)

In [110]:
# 查看df数据集
for features, labels in full_dataset.take(1):
    print("数据集格式验证:")
    print(f"  特征数量: {len(features)}")
    for name, tensor in features.items():
        print(f"  - {name}: shape={tensor.shape}, dtype={tensor.dtype}")
    print(f"  标签: shape={labels.shape}, dtype={labels.dtype}")
    break

数据集格式验证:
  特征数量: 12
  - hour: shape=(256,), dtype=<dtype: 'int32'>
  - weekday: shape=(256,), dtype=<dtype: 'int32'>
  - user_watch_stk_code_hash: shape=(256, 5), dtype=<dtype: 'int32'>
  - country_hash: shape=(256,), dtype=<dtype: 'int32'>
  - prefer_bid_code_hash: shape=(256, 5), dtype=<dtype: 'int32'>
  - hold_bid_code_hash: shape=(256, 5), dtype=<dtype: 'int32'>
  - user_propernoun_hash: shape=(256, 5), dtype=<dtype: 'int32'>
  - push_title_hash: shape=(256,), dtype=<dtype: 'int32'>
  - title_len: shape=(256,), dtype=<dtype: 'int32'>
  - item_code_hash: shape=(256, 5), dtype=<dtype: 'int32'>
  - submit_type_hash: shape=(256,), dtype=<dtype: 'int32'>
  - tag_id_hash: shape=(256, 3), dtype=<dtype: 'int32'>
  标签: shape=(256,), dtype=<dtype: 'int32'>


### 2.7 模型结构定义

#### 2.7.1 Embedding层

- class FeaturePipelineBuilder:
    实现了对 sparse；varlen_sparse；dense 三类特征的Embedding原子操作（基于tf2）
    定义并返回一个新的特征embedding的pipeline
- def process_feature_batch(features_dict: dict, pipelines: list) -> list:
    原子操作合并成process函数作用与原始数据（这里就是前向传播的时候call()流过来的数据）

In [103]:
class FeaturePipelineBuilder:
    """特征处理管道构建器 - 基于原项目实现"""
    
    def __init__(self, feat_configs: list, verbose: bool = True):
        self.feat_configs = feat_configs
        self.verbose = verbose
        self.embedding_layers = {}
        self.pooling_layers = {}
    
    def build_feature_pipelines(self) -> list:
        """构建特征处理管道"""
        pipelines = []
        
        for config in self.feat_configs:
            feat_name = config.get('feat_name')
            feat_type = config.get('feat_type')
            
            if not feat_name or not feat_type:
                continue
                
            # 创建处理器序列
            processors = self._create_processors(config)
            if processors:
                pipelines.append((feat_name, processors))
        
        if self.verbose:
            print(f"成功构建 {len(pipelines)} 个特征处理管道")
            for feat_name, processors in pipelines:
                processor_names = [p.__class__.__name__ for p in processors]
                print(f"  {feat_name}: {' -> '.join(processor_names)}")
        
        return pipelines
    
    def _create_processors(self, config: dict) -> list:
        """根据特征类型创建处理器"""
        feat_name = config['feat_name']
        feat_type = config['feat_type']
        vocab_size = config.get('vocabulary_size', 1000)
        embed_dim = config.get('embedding_dim', 8)
        
        if feat_type == 'sparse':
            embedding = Embedding(
                input_dim=vocab_size,
                output_dim=embed_dim,
                name=f'{feat_name}_embedding',
                mask_zero=False
            )
            return [embedding]
        
        elif feat_type == 'varlen_sparse':
            embedding = Embedding(
                input_dim=vocab_size,
                output_dim=embed_dim,
                name=f'{feat_name}_embedding',
                mask_zero=True  # 变长特征需要masking
            )
            pooling = GlobalAveragePooling1D(name=f'{feat_name}_pooling')
            return [embedding, pooling]
        
        elif feat_type == 'dense':
            # 密集特征直接通过，可以加BN
            identity = tf.keras.layers.Lambda(lambda x: x, name=f'{feat_name}_identity')
            return [identity]
        
        return []

def process_feature_batch(features_dict: dict, pipelines: list) -> list:
    """处理特征批次数据"""
    outputs = []
    
    for feat_name, processors in pipelines:
        if feat_name not in features_dict:
            continue
        
        # 依次应用处理器
        feature_input = features_dict[feat_name]
        for processor in processors:
            feature_input = processor(feature_input)
        
        outputs.append(feature_input)
    
    return outputs
    """准备TensorFlow数据集用于深度模型训练"""
    
    # 获取需要的特征 - 适配不同的配置文件格式
    if 'pipelines' in feat_config:
        # feat.yml 格式
        pipelines = feat_config['pipelines']
    elif 'process' in feat_config and 'pipelines' in feat_config['process']:
        # config.yml 格式
        pipelines = feat_config['process']['pipelines']
    else:
        raise ValueError("无法找到特征配置中的 pipelines")
    
    pipeline_feats = {p['feat_name']: p for p in pipelines}
    features_dict = {}
    
    for feat_name, config in pipeline_feats.items():
        if feat_name not in df.columns:
            continue
        
        feat_type = config.get('feat_type', 'sparse')
        
        if feat_type == 'sparse':
            # 单值特征，shape为 (batch_size,) - 这是关键！
            values = df[feat_name].values.astype(np.int32)
            features_dict[feat_name] = values
            
        elif feat_type == 'varlen_sparse':
            # 变长特征，需要padding
            sequences = df[feat_name].tolist()
            max_len = max(len(seq) if isinstance(seq, list) else 1 for seq in sequences)
            
            padded_sequences = []
            for seq in sequences:
                if isinstance(seq, list):
                    padded = seq + [0] * (max_len - len(seq))
                else:
                    padded = [int(seq)] + [0] * (max_len - 1)
                padded_sequences.append(padded[:max_len])
            
            features_dict[feat_name] = np.array(padded_sequences, dtype=np.int32)
            
        elif feat_type == 'dense':
            # 密集特征
            values = df[feat_name].values.astype(np.float32).reshape(-1, 1)
            features_dict[feat_name] = values
    
    # 准备标签
    labels = df['label'].values.astype(np.int32)
    
    # 创建数据集
    dataset = tf.data.Dataset.from_tensor_slices((features_dict, labels))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return dataset

In [111]:
if 'pipelines' in deep_feat_config:
    deep_pipelines = deep_feat_config['pipelines']
elif 'process' in deep_feat_config and 'pipelines' in deep_feat_config['process']:
    deep_pipelines = deep_feat_config['process']['pipelines']
else:
    raise ValueError("无法找到深度模型特征配置中的 pipelines")

#### 2.7.2 模型构建（Embedding层 + MLP）

In [112]:
class DeepMLP(tf.keras.Model):
    """深度MLP模型 - 基于原项目实现"""
    
    def __init__(self, feat_configs: list, train_config: dict = None, verbose: bool = True):
        super(DeepMLP, self).__init__()
        
        # 构建特征处理管道
        pipeline_builder = FeaturePipelineBuilder(feat_configs, verbose=verbose)
        self.feature_pipelines = pipeline_builder.build_feature_pipelines()
        
        # 特征连接层
        self.concat_layer = Concatenate(axis=1)
        
        # 获取模型参数 - 从train.yml动态读取
        model_params = (train_config or {}).get('model', {})
        hidden_layers = model_params.get('layers', [128, 64, 32])  # 使用train.yml的默认值
        dropout_rates = model_params.get('dropout_rates', [0.3, 0.3, 0.2])  # 使用train.yml的默认值
        l2_reg = model_params.get('l2_regularization', 0.001)
        
        # 构建分类器
        self.classifier = self._build_classifier(hidden_layers, dropout_rates, l2_reg)
    
    def _build_classifier(self, hidden_layers: list, dropout_rates: list, l2_reg: float):
        """构建分类器网络"""
        layers = []
        
        # 添加BatchNorm
        layers.append(BatchNormalization())
        
        # 添加隐藏层
        for i, units in enumerate(hidden_layers):
            layers.append(Dense(
                units, 
                activation='relu',
                kernel_regularizer=l2(l2_reg)
            ))
            if i < len(dropout_rates):
                layers.append(Dropout(dropout_rates[i]))
        
        # 输出层
        layers.append(Dense(1, activation='sigmoid', kernel_regularizer=l2(l2_reg)))
        
        return tf.keras.Sequential(layers)
    
    def call(self, features, training=None):
        """前向传播"""
        # 处理所有特征
        processed_outputs = process_feature_batch(features, self.feature_pipelines)
        
        if not processed_outputs:
            raise ValueError("没有有效的特征输出")
        
        # 合并特征
        if len(processed_outputs) > 1:
            concat_features = self.concat_layer(processed_outputs)
        else:
            concat_features = processed_outputs[0]
        
        # 应用分类器
        predictions = self.classifier(concat_features, training=training)
        return predictions

In [113]:
deep_model = DeepMLP(deep_pipelines, train_config, verbose=True)

成功构建 12 个特征处理管道
  hour: Embedding
  weekday: Embedding
  user_watch_stk_code_hash: Embedding -> GlobalAveragePooling1D
  country_hash: Embedding
  prefer_bid_code_hash: Embedding -> GlobalAveragePooling1D
  hold_bid_code_hash: Embedding -> GlobalAveragePooling1D
  user_propernoun_hash: Embedding -> GlobalAveragePooling1D
  push_title_hash: Embedding
  title_len: Embedding
  item_code_hash: Embedding -> GlobalAveragePooling1D
  submit_type_hash: Embedding
  tag_id_hash: Embedding -> GlobalAveragePooling1D


#### 2.7.3 编译模型

In [114]:
# 3. 编译模型
learning_rate = train_config.get('training', {}).get('lr', 0.0005)  # 使用train.yml的默认值
deep_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=[
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.BinaryAccuracy(name='accuracy')
    ]
)

### 2.8 模型训练

#### 2.8.1 数据集划分

In [115]:
validation_split = train_config.get('training', {}).get('validation_split', 0.2)
train_size = int((1 - validation_split) * len(df_deep_processed))

print(f"\n📊 数据使用情况:")
print(f"  总数据量: {len(df_deep_processed):,}")
print(f"  训练集: {train_size:,} ({(1-validation_split)*100:.1f}%)")
print(f"  验证集: {len(df_deep_processed)-train_size:,} ({validation_split*100:.1f}%)")
print(f"  训练批次数: {train_size // batch_size}")
print(f"  验证批次数: {(len(df_deep_processed)-train_size) // batch_size}")

train_dataset = full_dataset.take(train_size // batch_size)
val_dataset = full_dataset.skip(train_size // batch_size)


📊 数据使用情况:
  总数据量: 50,000
  训练集: 40,000 (80.0%)
  验证集: 10,000 (20.0%)
  训练批次数: 156
  验证批次数: 39


#### 2.8.2 模型训练

In [116]:
epochs = train_config.get('training', {}).get('epochs', 2)  # 使用train.yml的默认值
print(f"\n开始训练深度模型... (epochs={epochs})")

history = deep_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    verbose=1
)

# 6. 输出结果
print("\n--- 深度模型训练完成 ---")
if 'val_auc' in history.history:
    final_train_auc = history.history['auc'][-1]
    final_val_auc = history.history['val_auc'][-1]
    print(f"最终训练集 AUC: {final_train_auc:.4f}")
    print(f"最终验证集 AUC: {final_val_auc:.4f}")
    print(f"AUC差异: {final_train_auc - final_val_auc:.4f}")
else:
    final_auc = history.history['auc'][-1]
    print(f"最终 AUC: {final_auc:.4f}")

print(f"深度模型架构总结:")
print(f"特征处理管道: {len(deep_model.feature_pipelines)} 个")
print(f"分类器层数: {len([l for l in deep_model.classifier.layers if isinstance(l, Dense)])}")
print(f"总参数量: {deep_model.count_params():,}")

# 显示深度模型训练结果
print(f"深度模型训练结果:")
if 'val_auc' in history.history:
    print(f"验证集AUC: {final_val_auc:.4f}")
    print(f"训练集AUC: {history.history['auc'][-1]:.4f}")
else:
    print(f"AUC: {final_auc:.4f}")


开始训练深度模型... (epochs=2)
Epoch 1/2
156/156 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.7465 - auc: 0.7228 - loss: 0.7396 - val_accuracy: 0.8024 - val_auc: 0.8264 - val_loss: 0.6571
Epoch 2/2
156/156 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7938 - auc: 0.8093 - loss: 0.6395 - val_accuracy: 0.8044 - val_auc: 0.8282 - val_loss: 0.5839

--- 深度模型训练完成 ---
最终训练集 AUC: 0.8203
最终验证集 AUC: 0.8282
AUC差异: -0.0079
深度模型架构总结:
特征处理管道: 12 个
分类器层数: 4
总参数量: 505,417
深度模型训练结果:
验证集AUC: 0.8282
训练集AUC: 0.8203
